In [1]:
import os
import mido
import datetime
import pretty_midi
import mir_eval.display
import librosa.display
import matplotlib.pyplot as plt
import IPython.display
import numpy as np
import pandas as pd
from mido import MidiFile
from mido import MetaMessage
from mido import tempo2bpm, bpm2tempo

In [31]:
# 左右手音符都在同一個track，可以用channel=0或channel=1去判斷
def analyzemidi(path):
    mid = MidiFile(path)
    txt = path.split('/')
    midiname = txt[-1]

    count1 = 0
    count2 = 0
    final_R_list = []
    final_L_list = []
    
    for i, track in enumerate(mid.tracks):
        print('Track {}: {}'.format(i, midiname))
        passed_time = 0
        passed_time2 = 0

        for info in track:
            if info.type == "set_tempo":
                tempo = info.tempo
                print("Tempo:", tempo)
                print("Bpm:", tempo2bpm(info.tempo))
                print('Ticks_per_beat:', mid.ticks_per_beat)
                print()

        for msg in track:
            '''
            if msg.type == "note_on":
                print(msg)
            #print(count)
            '''
            
            # 把tick單位轉成second單位
            ab_time = mido.tick2second(msg.time, mid.ticks_per_beat, tempo)

            y = ab_time

            # real_time是每一個事件在整個midi文件中的真實時間位置
            real_time = ab_time + passed_time
            passed_time += ab_time
            R_list = []
            
            # channel == 0右手、channel == 1左手
            if msg.type == "note_on":
                channel = msg.channel
                velocity = msg.velocity
                note = msg.note
                # 右手
                if channel == 0:
                    x = passed_time
                    print("note = {0:3}  |  real time = {1:<8}".format(note, round(real_time, 3)))
                    R_list.append(note)
                    R_list.append(real_time)
                    count1 += 1
                    final_R_list.append(R_list)
        
        if count1 != 0:
            print('Midi Length (R):', real_time)
            print('Count1:', count1)
            print()
        
        for msg in track:
            #print(msg)
            #print(count)

            # 把tick單位轉成second單位
            ab_time2 = mido.tick2second(msg.time, mid.ticks_per_beat, tempo)

            y2 = ab_time2

            # real_time是每一個事件在整個midi文件中的真實時間位置
            real_time2 = ab_time2 + passed_time2
            passed_time2 += ab_time2
            L_list = []
            
            # channel == 0右手、channel == 1左手
            if msg.type == "note_on":
                channel = msg.channel
                velocity = msg.velocity
                note = msg.note
                # 左手
                if channel == 1:
                    x2 = passed_time2
                    print("note = {0:3}  |  real time = {1:<8}".format(note, round(real_time2, 3)))
                    L_list.append(note)
                    L_list.append(real_time2)
                    count2 += 1
                    final_L_list.append(L_list)
        
        if count2 != 0:
            print('Midi Length (L):', real_time2)
            print('Count2:', count2)
    
    return final_R_list,final_L_list

In [34]:
path = 'mid/ABBA_Andante Andante midi download.mid'
final_R_list,final_L_list = analyzemidi(path)

Track 0: ABBA_Andante Andante midi download.mid
Tempo: 582520
Bpm: 103.00075533887248
Ticks_per_beat: 384

note =  57  |  real time = 2.039   
note =  60  |  real time = 2.33    
note =  65  |  real time = 2.621   
note =  60  |  real time = 2.913   
note =  57  |  real time = 3.204   
note =  81  |  real time = 3.301   
note =  79  |  real time = 3.398   
note =  55  |  real time = 3.786   
note =  60  |  real time = 4.078   
note =  76  |  real time = 4.369   
note =  60  |  real time = 4.66    
note =  74  |  real time = 5.243   
note =  58  |  real time = 5.825   
note =  72  |  real time = 6.019   
note =  62  |  real time = 6.116   
note =  74  |  real time = 6.116   
note =  72  |  real time = 6.408   
note =  53  |  real time = 6.699   
note =  70  |  real time = 6.699   
note =  53  |  real time = 7.281   
note =  77  |  real time = 7.281   
note =  57  |  real time = 7.573   
note =  60  |  real time = 7.864   
note =  57  |  real time = 8.155   
note =  53  |  real time = 8.

In [46]:
# 毫秒單位轉換(1秒-1000毫秒)
# https://stackoverflow.com/questions/775049/how-do-i-convert-seconds-to-hours-minutes-and-seconds
def convert_time(final_R_list,final_L_list):
    if len(final_R_list) == 0:
        #print('final_R_list empty ' + str(final_L_list[0]))
        realtime = final_L_list[0][1]
    if len(final_L_list) == 0:
        #print('final_L_list empty ' + str(final_R_list[0]))
        realtime = final_R_list[0][1]
    if len(final_R_list) != 0 and len(final_L_list) != 0:
        #print('Not empty ' + str(final_R_list[0]))
        if final_R_list[0][1] < final_L_list[0][1]: # 若同時左右手都有音，挑選最先到的音符
            realtime = final_R_list[0][1]
        else:
            realtime = final_L_list[0][1]
    
    space = str(datetime.timedelta(seconds=realtime))[2:11]
    return space

In [47]:
space = convert_time(final_R_list,final_L_list)
space

'00:01.747'

In [4]:
# 查看midi檔案總長度(回傳MIDI檔案中最後一個元素的時間值)
pm = pretty_midi.PrettyMIDI(path)
time = pm.get_end_time()
print('midi檔案總長度:',time)

midi檔案總長度: 268.25046000000003
